In [ ]:
import os

import torch
from ipywidgets import interact
from qsr_learning.data.data import draw, generate_objects

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Test Relations

In [ ]:
from qsr_learning.relations.absolute.half_planes import above, below, left_of, right_of


@interact(x1=(0, 150), y1=(0, 150), x2=(0, 150), y2=(0, 150))
def test_spatial_relations(x1, y1, x2, y2):
    object1, object2 = generate_objects(
        positions=[(y1, x1), (y2, x2)], image_size=(32, 32), crop=True, bbox=True
    )
    display(draw([object1, object2]))
    print(object1.bbox.top, object1.bbox.bottom, object1.bbox.left, object1.bbox.right)
    print(object2.bbox.top, object2.bbox.bottom, object2.bbox.left, object2.bbox.right)
    print(
        "left_of({}, {}): {}".format(
            object1.name, object2.name, left_of(object1, object2)
        )
    )
    print(
        "right_of({}, {}): {}".format(
            object1.name, object2.name, right_of(object1, object2)
        )
    )
    print(
        "above({}, {}): {}".format(object1.name, object2.name, above(object1, object2))
    )
    print(
        "below({}, {}): {}".format(object1.name, object2.name, below(object1, object2))
    )

# Test Negative Sample Generation

In [ ]:
import numpy as np
from munch import Munch
import random

from qsr_learning.data.data import generate_one_negative_example

In [ ]:
def test_negative_sample_generation():
    object_names = {"A", "B", "C"}
    relation_names = {"0", "1"}
    positive_examples = {("A", "0", "B")}

    mixture = Munch(head=1, relation=1, tail=1)
    negative_sample_types = list(mixture.keys())
    p = np.array(list(mixture.values()))
    p = p / p.sum()

    negative_examples = []

    for _ in range(10):
        negative_sample_type = np.random.choice(negative_sample_types, p=p)
        negative_example = generate_one_negative_example(
            object_names, relation_names, positive_examples, negative_sample_type
        )
        negative_examples.append(negative_example)

    return negative_examples

test_negative_sample_generation()

# Test a Trained Model

In [ ]:
def test_trained_model(model):
    def tensor2image(x):
        return Image.fromarray(
            (255 * ((0.5 * x) + 0.5)).numpy().astype("uint8").transpose(1, 2, 0)
        )

    def tensor2question(x):
        return [loader.train.dataset.idx2word[idx] for idx in x.numpy()]

    def tensor2answer(x):
        return x.item()

    for batch in loader.test:
        break

    images, questions, answers = (item.to(device) for item in batch)
    with torch.no_grad():
        model.eval()
        print(model(images[:1], questions[:1]) > 0.5)
        print(answers[:1])

    for i in range(batch[0].shape[0]):
        display(tensor2image(images[i].cpu()))
        display(tensor2question(questions[i].cpu()))
        display(tensor2answer(answers[i].cpu()))